In [1]:
# Import the modules
import numpy as np
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_data = pd.read_csv("Resources/lending_data.csv")

# Review the DataFrame
lending_data.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the y variable, the labels
y = lending_data["loan_status"]
target_names = ["healthy", "high-risk"]

# Separate the X variable, the features
X = lending_data.drop(columns="loan_status")

In [4]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

In [7]:
y.shape

(77536,)

In [8]:
X.shape

(77536, 7)

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [9]:
# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.28, random_state=1)

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [10]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lr_model = LogisticRegression(solver="lbfgs",random_state=1)
# Fit the model using training data
logistic_regression_model = lr_model.fit(X_train, y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [11]:
# Make a prediction using the testing data
y_prediction = logistic_regression_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [12]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, y_prediction)

0.9504856484588143

In [13]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, y_prediction)

array([[20908,   110],
       [   65,   628]], dtype=int64)

In [14]:
# Print the classification report for the model
print(classification_report(y_test, y_prediction, target_names = target_names))

              precision    recall  f1-score   support

     healthy       1.00      0.99      1.00     21018
   high-risk       0.85      0.91      0.88       693

    accuracy                           0.99     21711
   macro avg       0.92      0.95      0.94     21711
weighted avg       0.99      0.99      0.99     21711



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** It predicts the healthy loan very well, but struggles to predict the high-risk loan. This is not very helpful for trying to identify high-risk loans

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [15]:
# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resample, y_resample = ros.fit_resample(X_train, y_train)

In [16]:
original_count = Counter(y_train).items()
original_count

dict_items([(0, 54018), (1, 1807)])

In [17]:
# Count the distinct values of the resampled labels data
resample_count = Counter(y_resample).items()
resample_count

dict_items([(0, 54018), (1, 54018)])

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [18]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
resample_model = LogisticRegression(solver="lbfgs",random_state=1)
# Fit the model using the resampled training data
resample_model.fit(X_resample, y_resample)

# Make a prediction using the testing data
resample_prediction = resample_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [19]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, resample_prediction)

0.9941403554734984

In [20]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, resample_prediction)

array([[20893,   125],
       [    4,   689]], dtype=int64)

In [21]:
# Print the classification report for the model
print(classification_report(y_test, resample_prediction, target_names = target_names))

              precision    recall  f1-score   support

     healthy       1.00      0.99      1.00     21018
   high-risk       0.85      0.99      0.91       693

    accuracy                           0.99     21711
   macro avg       0.92      0.99      0.96     21711
weighted avg       0.99      0.99      0.99     21711



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The oversampled model improved balanced accuracy and recall by better capturing high-risk loans